In [2]:
from pathlib import Path
import seaborn as sns
import os
import json

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

## nice trick to increase dpi of exported images -> higher quality png's for presentation
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', quality=100)

# Objective

Explore the effects of supply-side shocks on the U.S. Economy through the lens of graph networks

# Defenitions

- **Supply tables:** The value of goods and services available in the U.S. economy, whether produced by domestic industries or imported. 
- **Use(IO) tables:** How the supply of goods and services is used. Includes purchases by U.S. industries, individuals, and government, and exports to foreign purchasers. This is the table we are actually interested in because $a_{ij}x_{j}$ is the total number of units (Millions of Dollars)

When it comes to the application of probabilsitc graph models to societal concerns, one need not look further than into public economic policy.

1. **Open Model:**
2. **Closed Model:**

In fact, these methods have proved particularly effective in the analysis of sudden and large changes, as in the case of military mobilization or other far-reaching tranformations of an economy. The method has also been applied in studies of how cost and price changes are transmitted through various sectors of an economy.

Among recent developments of the method may be mentioned its extension to include residuals of the production system – smoke, water pollution, scrap, etc., and the further processing of these. In this way the effects of the production on the environment can be studied.

# Setup

**Required:**
1. reqs_master $\coloneqq$ Total Requirements Table from 1997-2020 from BEA database
2. mapping.json $\coloneqq$ Dictionary containing shortened industry names
3. 'IO.py' $\coloneqq$ Python module containing IO class & functions for IO-dataframe -> networkX.G object

Please deploy the cells below and ensure that there are no errors; note that file paths may need to be changed to the directory where the data is stored on your local machine. More information on the data, mapping dictionary, and IO module may be found in `EDA-Andrew.ipynb`, where the function is built and we conduct a prelimiary EDA with visualzations.

In [3]:
## import BEA total requirements table
data_path = f'{str(Path(os.getcwd()).parent)}/data/processed'

# assign path

path, dirs, files = next(os.walk(f'{data_path}/../io-reqs-table'))
file_count = len(files)
files.sort()

# create empty list
reqs_master = []
  
# append datasets to the list 
for i in range(file_count):
    temp_df = pd.read_csv(path + "/" + files[i], sep=',', 
                skiprows = 4, header = [1]).iloc[:72,1:].set_index('Industry Description')
                
    reqs_master.append(temp_df)

In [4]:
## import our mapping.json dictionary 
with open("./../data/dict.json", 'r') as myfile:
    data=myfile.read()
    
mapping = json.loads(data)

In [5]:
## import IO module containing IO() class and pre-built functions from EDA notebook
from IO import IO
io = IO.IO()
io ## sanity check

# Model Implementation & Deployment

From a big picture perspective, we will be implementing a variety of proposed Economic Network models in published scientific journals. We will be starting with a relatively simple model porposed by .

From the algebra, one could easily mistake this for a trivial task; however, remember that our IO tables are of sizes. The following concerns must be investigated and adressed:

- How can we guarantee the nonsigularity of our input-output matrix A? 
- Should we utilize the psuedoinverse and how will taht affect the predictive power of our model? 
- How do we deal with the computational complaexity of possible inverting a ~100x100 matrix? 

For those of you interested, these articles may be found both linked in the model summaries below or at the end in the 'References' section. 

## Simulation 1: Shock to Primary Metals Industry

We will first be attempting to fit underpowered the model proposed by . This model does not include .

If Leontif's economic model was good enough for a Nobel prize, let us assume that it is sufficient for this project.

- A = $\sigma$
- dZ = induced shock vector
- dY = log(output_change)

In [11]:
## simualted shock of primary metals industry - 2020
df = reqs_master[23]
H = df.copy().iloc[:71,:]

## 1x72 matrix where only nonzero element is on the index corresponding to 'primary metals' column
z = [0]*71
z[9] = -0.6

res = H * z

In [12]:
## largest decrease in log output
res['Primary metals'].sort_values().head(6)

Industry Description
Primary metals                                                   -0.845561
Wholesale trade                                                  -0.122471
Mining, except oil and gas                                       -0.042747
Fabricated metal products                                        -0.040744
Miscellaneous professional, scientific, and technical services   -0.038364
Truck transportation                                             -0.031992
Name: Primary metals, dtype: float64

## Simulation 2

Next, Acemogule et al. suggests that the inclusion of taxation and the federal government as an industry in the overall economic network. **In fact, one could argue that the method proposed by John B. Long and Acemoglu are simply a variant of the closed Leotnitz model.**


In [13]:
H

,Farms,"Forestry, fishing, and related activities",Oil and gas extraction,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Wood products,Nonmetallic mineral products,Primary metals,...,"Performing arts, spectator sports, museums, and related activities","Amusements, gambling, and recreation industries",Accommodation,Food services and drinking places,"Other services, except government",Federal general government (defense),Federal general government (nondefense),Federal government enterprises,State and local general government,State and local government enterprises
Industry Description,,,,,,,,,,,,,,,,,,,,,
Farms,1.231440,0.023236,0.001630,0.002204,0.001139,0.000738,0.003032,0.016062,0.002448,0.001802,...,0.001973,0.016962,0.007405,0.020431,0.001796,0.001956,0.000365,0.002272,0.008419,0.001545
"Forestry, fishing, and related activities",0.072096,1.110935,0.000673,0.001968,0.000678,0.000416,0.003996,0.124927,0.001709,0.001853,...,0.000466,0.003521,0.003651,0.006497,0.000917,0.000544,0.000266,0.000820,0.002786,0.001282
Oil and gas extraction,0.016900,0.005253,1.126379,0.019905,0.010190,0.056173,0.016293,0.009440,0.008337,0.012857,...,0.002513,0.006588,0.005687,0.006096,0.004113,0.009667,0.001525,0.020378,0.019553,0.073081
"Mining, except oil and gas",0.008593,0.000950,0.006151,1.076815,0.002857,0.016257,0.016351,0.003116,0.066017,0.071245,...,0.002314,0.003989,0.001902,0.002185,0.001936,0.001494,0.001570,0.001342,0.002423,0.021009
Support activities for mining,0.000766,0.000200,0.036456,0.024344,1.012442,0.002190,0.000993,0.000431,0.001836,0.002105,...,0.000140,0.000316,0.000246,0.000264,0.000194,0.000371,0.000093,0.000708,0.000698,0.002884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Federal general government (defense),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
Federal general government (nondefense),0.006179,0.002277,0.008481,0.004052,0.009372,0.003985,0.003999,0.004833,0.004435,0.007195,...,0.003214,0.003794,0.004162,0.004536,0.003455,0.012272,1.008245,0.008403,0.001906,0.003662
Federal government enterprises,0.002823,0.000901,0.002605,0.002898,0.002524,0.002783,0.001880,0.002352,0.002456,0.003710,...,0.001516,0.001883,0.004246,0.003984,0.002550,0.001050,0.001224,1.001350,0.001611,0.002346


# Visualizations

This section will contain a couple of fancy network plots created to accompany the analysis and write-ups due March 17, 2022.

# Analysis

# Conclusion

## Limitations

## Next Steps

As stated in the model portion above,

# References
1. Acemoglu, D., Akcigit, U., & Kerr, W. (2016). Networks and the macroeconomy: An empirical exploration. NBER Macroeconomics Annual, 30(1), 273–335. https://doi.org/10.1086/685961 
2. Bureau of Economic Analysis. (n.d.). Total Requirements Derivation. bea.gov. Retrieved March 17, 2022, from https://apps.bea.gov/industry/pdf/TotalRequirementsDerivation.pdf 
3. Choi, Jason and Forester, Andrew, (2017), The Changing Input-Output Network Structure of the U.S. Economy, Economic Review, (Q II), 23-49
4. Karen J. Horowitz & Mark A. Planting, 2006. "Concepts and Methods of the U.S. Input-Output Accounts," BEA Papers 0066, Bureau of Economic Analysis.
5. Sekhon, Rupinder, and Roberta Bloom. Applications – Leontief Models. De Anza College, 4 Sept. 2021, https://math.libretexts.org/@go/page/37851.